# 第二课
> 项目代码：https://github.com/RanFeng/NumpyFlow

## 提升开发效率

> 工欲善其事，必先利其器

- 使用jupyter

``` bash
jupyter-notebook
``` 

- 在jupyter中使用自己的conda环境

``` bash
conda activate py37
conda install nb_conda_kernels
```

- shell小技巧

```
alias ca="conda activate"
```




In [3]:
print("sadasdas")

sadasdas


In [34]:
import numpy as np
import nf
import torch as th

In [50]:
a = nf.Tensor([[1, 2, 3]], requires_grad=True)
b = nf.Tensor([[5], [5]], requires_grad=True)
c = a * b

def broadcastable(grad, ashape):
    """
    保证传递的梯度shape一致，用于兼容广播机制的反向传播
    :param grad:
    :param ashape:
    :return:
    """
    if grad.shape == ashape:
        return grad
    grad_bak = grad.sum(axis=tuple(range(grad.ndim - len(ashape))))
    # print("g", grad_bak.shape, ashape)
    keepdims = tuple(n for (n, i) in enumerate(grad_bak.shape) if i != ashape[n])
    if keepdims:
        grad_bak = grad_bak.sum(axis=keepdims, keepdims=True)
    # print("g", grad_bak.shape, ashape)
    return grad_bak

In [85]:
# [[1,2,3], [1,2,3]]
# [[5,5,5], [5,5,5]]
A = th.Tensor(np.array([[1,2,3],[4,5,6]]))
B = th.Tensor(np.array([100, 200, 300]))
c = A + B
# result = A + [100, 200, 300]

# grad = np.array([[[1,1,1], [1,1,1]]])
# d = nf.Tensor(grad)
# print(d.shape)
# print(d.T.shape)
# print(grad.shape)
# print(grad.T.shape)
# c.backward(1)

# grad_bak = broadcastable(grad, (2, 1))
print(c)
print(A.grad)
print(B.grad)

# print(a.shape, a.grad)
# print(b.shape, b.grad)

tensor([[101., 202., 303.],
        [104., 205., 306.]])
None
None


In [29]:
th.abs(th.Tensor([[[-1,2,0,1,2]]])).T.shape

torch.Size([5, 1, 1])

In [89]:
def func(x,y,z):
    f6 = x + y + 1 * x * y 
    return f6
def nf_grad_Test(x,y,z):
    x = nf.Tensor(x, requires_grad=True) # grad
    y = nf.Tensor(y, requires_grad=True)
    z = nf.Tensor(z, requires_grad=True)
    f9 = func(x,y,z)
    f9.backward() # 计算梯度
#     print("nf", t2,time() - t1)
    return [x.grad, y.grad, z.grad]
x = np.array([5])
y = np.array([2])
z = np.array([3])
grad_nf = nf_grad_Test(x,y,z)
print(grad_nf)

[array([3.]), array([6.]), None]


f(x,y) = x + y
f/x=1   f/y=1

In [56]:

class Add:
    def __call__(self, a, b):
        self.var = (a, b)
        return a + b
    
    def backward(self, grad):
        self.var[0].backward(grad)
        self.var[1].backward(grad)

class Assign :
    def __call__(self, a):
        self.var = (a)
        return a

    def backward(self, grad):
        return None
        
class Mul:
    def __call__(self, a, b):
        self.var = (a, b)
        return a.data * b.data
    
    def backward(self, grad):
        self.var[0].backward(grad*b.data)
        self.var[1].backward(grad*a.data)





class TestTensor :
    def __init__(self, data=None, creator=None):
        self.data = None
        if isinstance(data, (int, float, bool)):
            data = [data]
        if isinstance(data, (list, tuple)):
            data = np.array(data)
        if isinstance(data, (np.ndarray)):
            self.data = data.copy()
        elif isinstance(data, TestTensor):
            raise ValueError("输入的是 TestTensor")
        else:
            raise ValueError("输入类型未知", type(data), data)
        if creator is None:
            creator = Assign()
            creator(self)
        self.creator = creator
        self.grad = self.data * 0
        
    def __mul__(self, other): # 乘法
        mul_op = Mul()
        return TestTensor(self * other, mul_op)
    def __rmul__(self, other): # 乘法
        mul_op = Mul()
        return TestTensor(mul_op(other, self), mul_op)
    def backward(self, grad):
        self.grad += grad
        if self.creator:
            self.creator.backward(grad)

In [64]:
a = TestTensor(np.array([[6, 6], [1, 2]]))
b = TestTensor([[2, 2], [3, 4]])
# c = TestTensor()
e = a * b
print(e.data)

# e.backward(1)
# print(a.grad)
# print(b.grad)
print(e)

[[12 12]
 [ 3  8]]


In [33]:
a = np.array([[1, 2, 3], [4,5,6]])
b = np.array([4])

# print(a, a.shape) # 广播机制
print(a.reshape([3,2]))

# Reshape(a, [3,2])

[[1 2 3]
 [4 5 6]] (2, 3)
[[1 2]
 [3 4]
 [5 6]]


## 作业
1. 了解numpy的广播机制
2. 了解广播机制的反向传播

## 参考
- https://segmentfault.com/a/1190000023346483
- https://www.heywhale.com/mw/project/59f29f67c5f3f5119527a2cc

---